# Loading and calculating returns in LUSID

This notebook shows how you can load and calculate returns in LUSID. In the cells below, we load a CSV file of daily returns over a 5 year period (approximately). We then use LUSID to calculate various return metrics:

* Daily
* Since inception
* Week-to-date (WTD), Month-to-date (MTD), Quarter-to-date (QTD), Year-to-date (YTD)
* 1 month (1M), 3 months (3M), 1 Year (1Y)
* Since a specific date
* Annualised since incorporation
* 5 year annualised


# Setup LUSID

In [ ]:
# Import common libraries
import os
import pandas as pd
import logging
import pytz
import json
import random
import numpy as np
from datetime import datetime, timezone
from IPython.core.display import HTML
logging.basicConfig(level = logging.INFO)

# Import LUSID libraries
import lusid as lu
import lusid.models as lm

import lusidjam
import lusid.extensions as le
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from finbourne_sdk_utils.lpt.lpt import to_date
from finbourne_sdk_utils import cocoon as cocoon

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    le.ArgsConfigurationLoader(access_token = lusidjam.RefreshingToken(), app_name = "LusidJupyterNotebook"),
    le.EnvironmentVariablesConfigurationLoader(),
    le.SecretsFileConfigurationLoader(secrets_path)]
api_factory = le.SyncApiClientFactory(config_loaders=config_loaders)

# Confirm success
api_client = api_factory.build(lu.ApplicationMetadataApi)
api_url = api_client.api_client.configuration._base_path.replace("api","")

print ('LUSID Environment :', api_url + "docs")
display(pd.DataFrame(api_client.get_lusid_versions().to_dict()))

In [ ]:
transaction_portfolios_api = api_factory.build(lu.TransactionPortfoliosApi)
portfolios_api = api_factory.build(lu.PortfoliosApi)

# Setup defaults

In [ ]:
scope = "returns"
portfolio_code = "UK_EQUITY"

start_date = datetime(year=2020, month=1, day=2, tzinfo=pytz.UTC)
number_of_days = 366

from_effective_at = datetime(year=2013, month=12, day=31, tzinfo=pytz.utc)
to_effective_at = datetime(year=2020, month=5, day=3, tzinfo=pytz.utc)

In [ ]:
def parse_csv_date(date_str):
    try:
        dt = datetime.strptime(date_str, '%d/%m/%Y')
    except ValueError:
        # Fall back to 2-digit year\n",
        dt = datetime.strptime(date_str, '%d/%m/%y')
    
    return dt.replace(tzinfo=timezone.utc)

# Create a portfolio

In [ ]:
try:
    response = transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            created=datetime(year=2013, month=12, day=30, tzinfo=pytz.UTC).isoformat(),
            base_currency="GBP"
        ),
    )
except lu.ApiException as e:
    print(json.loads(e.body)["title"])

# Upload a DataFrame of returns from CSV file to LUSID

In [ ]:
returns_df = pd.read_csv("data/returns/performance_data.csv", dtype={"daily_returns": np.int64})
#returns_df["date"] =  pd.to_datetime(returns_df["date"], format="%d/%m/%Y")
returns_df = returns_df.rename(
    columns={"mv": "market_value", "returns": "daily_returns"}
)

returns_df.head()

> <b> NOTE: We have truncated the upsert to 10 rows. Remove the cell below if you want to load all 2500+ rows of daily returns.</b>

In [ ]:
returns_df = returns_df[:10]

In [ ]:
time_series_of_returns = [
    lm.PerformanceReturn(
        effective_at=parse_csv_date(row["date"]),
        rate_of_return=row["daily_returns"],
        closing_market_value=row["market_value"],
        period="Daily",
    )
    for _, row in returns_df.iterrows()
]

return_code = "daily_return"

upsert_returns = portfolios_api.upsert_portfolio_returns(
    scope=scope,
    code=portfolio_code,
    return_code=return_code,
    return_scope=scope,
    performance_return=time_series_of_returns
)

print(f"Upserted {len(time_series_of_returns)} returns for {portfolio_code} in scope {scope} with return code {return_code}")

# Retrieve returns from LUSID

In [ ]:
returns = portfolios_api.get_portfolio_aggregated_returns(
    scope=scope,
    code=portfolio_code,
    aggregated_returns_request=lm.AggregatedReturnsRequest(
        metrics=[
            lm.PerformanceReturnsMetric(
                type="Return",
                window="1D",
                alias="1D"),
            lm.PerformanceReturnsMetric(
                type="Return",
                window="INC",
                alias="INC"),
            lm.PerformanceReturnsMetric(
                type="Return",
                window="WTD",
                alias="WTD"),
            lm.PerformanceReturnsMetric(
                type="Return",
                window="MTD",
                alias="MTD"),
            lm.PerformanceReturnsMetric(
                type="Return",
                window="QTD",
                alias="QTD"),
            lm.PerformanceReturnsMetric(
                type="Return",
                window="YTD",
                alias="YTD"),
            lm.PerformanceReturnsMetric(
                type="Return",
                window="1M-ROLLING",
                alias="1M-ROLLING"),
            lm.PerformanceReturnsMetric(
                type="Return",
                window="3M-ROLLING",
                alias="3M-ROLLING"),
            lm.PerformanceReturnsMetric(
                type="Return",
                window="1Y-ROLLING",
                alias="1Y-ROLLING"),
            lm.PerformanceReturnsMetric(
                type="Return",
                window="SINCE(2014-03-19)",
                alias="SINCE(2014-03-19)"),
            lm.PerformanceReturnsMetric(
                type="Return",
                window="INC",
                annualised=True,
                alias="ANNUALISED(INC)"),
            lm.PerformanceReturnsMetric(
                type="Return",
                window="5Y-ROLLING",
                annualised=True,
                alias="ANNUALISED(5Y-ROLLING)")
        ],
        return_ids=[lm.ResourceId(
            scope=scope,
            code=return_code)],
        period="Daily",
        outputFrequency="Daily",
    ),
    from_effective_at=from_effective_at.isoformat(),
    to_effective_at=to_effective_at.isoformat() ,
)

In [ ]:
data = returns.results[f"{scope}/{return_code}"]

summary_df = pd.DataFrame(
    columns=[
        "date",
        "opening_mv",
        "closing_mv",
        "day",
        "inception",
        "mtd",
        "ytd",
        "qtd",
        "wtd",
        "1m-rolling",
        "3m-rolling",
        "1y-rolling",
        "since_20140319",
        "annualised_inc",
        "annualised_5y-rolling",
    ]
)

def make_row(item):
    row = {}
    row["date"] = item.effective_at.strftime(format="%Y-%m-%d")
    row["opening_mv"] = item.opening_market_value
    row["closing_mv"] = item.opening_market_value
    row["day"] = item.metrics_value.get("1D")
    row["inception"] = item.metrics_value.get("INC")
    row["mtd"] = item.metrics_value.get("MTD")
    row["ytd"] = item.metrics_value.get("YTD")
    row["qtd"] = item.metrics_value.get("QTD")
    row["wtd"] = item.metrics_value.get("WTD")
    row["1m"] = item.metrics_value.get("1M-ROLLING")
    row["3m"] = item.metrics_value.get("3M-ROLLING")
    row["1y"] = item.metrics_value.get("1Y-ROLLING")
    row["since_20140319"] = item.metrics_value.get("SINCE(2014-03-19)")
    row["annualised_inc"] = item.metrics_value.get("ANNUALISED(INC)")
    row["annualised_5y"] = item.metrics_value.get("ANNUALISED(5Y-ROLLING)")
    return row


summary_df = pd.concat(
    [summary_df, pd.DataFrame((make_row(item) for item in data))]
).reset_index(drop=True)

summary_df.tail(10)

In [ ]:
def time_series_performance():
    ts_performance = summary_df.plot(y=["inception"], figsize=(12, 9))
    ts_performance.set_title(
        f"Performance of {portfolio_code} since inception",
        fontsize="large",
    )
    ts_performance.set_ylabel("Performance in %", fontsize="large")
    ts_performance.set_xlabel("Days since inception", fontsize="large")
    ts_performance.legend(prop={"size": 12})


time_series_performance()

# Returns Dashboard

To view the returns loaded above, navigate to the following link:

In [ ]:
display(HTML(f'<a href="{api_url}app/dashboard/returns?dateFrom={from_effective_at.isoformat()}&dateTo={to_effective_at.isoformat()}&frequency=Daily&compositeMethod=Asset&period=Daily&scope={scope}&code={portfolio_code}&entityType=Portfolio&returnsScope={scope}&returnsCode={return_code}" target="_blank">See Reference Portfolio Constituents</a>'))